### Import libraries

In [1]:
import pandas as pd
import plotly.graph_objects as go

### Load and process data

In [2]:
data = pd.read_csv('./covid-20200330.csv')
print(data.head())

      FIPS     Admin2  Province_State Country_Region          Last_Update  \
0  45001.0  Abbeville  South Carolina             US  2020-03-30 22:52:45   
1  22001.0     Acadia       Louisiana             US  2020-03-30 22:52:45   
2  51001.0   Accomack        Virginia             US  2020-03-30 22:52:45   
3  16001.0        Ada           Idaho             US  2020-03-30 22:52:45   
4  19001.0      Adair            Iowa             US  2020-03-30 22:52:45   

         Lat       Long_  Confirmed  Deaths  Recovered  Active  \
0  34.223334  -82.461707          3       0          0       0   
1  30.295065  -92.414197         11       1          0       0   
2  37.767072  -75.632346          6       0          0       0   
3  43.452658 -116.241552        113       2          0       0   
4  41.330756  -94.471059          1       0          0       0   

                    Combined_Key  
0  Abbeville, South Carolina, US  
1          Acadia, Louisiana, US  
2         Accomack, Virginia, US  


In [3]:
# sort by the number of confirmed cases
# in decreasing order.
data = data.sort_values(['Confirmed'], ascending=False)

# group records by country.
data = data.groupby('Country_Region').agg({ 'Lat': 'first'
                                           ,'Long_': 'first'
                                           ,'Confirmed': 'sum'
                                           ,'Deaths': 'sum'
                                           ,'Recovered': 'sum'})
data = data.reset_index()

# define a scaling function
# this function helps adjust the size of markers on the plot.
scale_factor = max(data['Confirmed'])
def scaling(column, factor):
    return (column / factor) ** 0.5 * 50

# make representations of the number of cases
# using the above-defined scaling function.
data['Confirmed-ref'] = scaling(data['Confirmed'], scale_factor)
data['Ended-ref']     = scaling(data['Deaths'] + data['Recovered'], scale_factor)
data['Death-ref']     = scaling(data['Deaths'], scale_factor)

# make a summary for each country
# this will be shown as we hover over the markers on the plot.
data['summary'] = 
    '<b>' + 'Country: ' + \
            data['Country_Region'] + '<br><br>' + \
    '</b>' + \
            'Confirmed: ' + \
            data['Confirmed'].astype(str) + '<br>' +\
            'Recovered: ' + \
            data['Recovered'].astype(str) + '<br>' +\
            'Deaths: ' + \
            data['Deaths'].astype(str)

In [6]:
# make a figure
fig = go.Figure()

# plot the number of confirmed cases.
fig.add_trace(go.Scattergeo(
         lat = data['Lat']
        ,lon = data['Long_']
        ,marker = dict(
            size = data['Confirmed-ref']
           ,color = data['Confirmed-ref'] 
           ,colorscale = [[0, 'rgb(255,255,0)'], [1, 'rgb(255,0,0)']]
           ,opacity = 1
                      )
        ,text = data['summary']
        ,hovertemplate = '%{text} <extra></extra>'
        ))

# draw the border of each country.
fig.update_geos(
    showcountries=True
)

# show figure
fig.show()

In [7]:
# make a figure
fig = go.Figure()

# plot the number of confirmed cases.
fig.add_trace(go.Scattergeo(
         lat = data['Lat']
        ,lon = data['Long_']
        ,marker = dict(
            size = data['Confirmed-ref']
           ,color = data['Confirmed-ref'] 
           ,colorscale = [[0, 'rgb(255,255,0)'], [1, 'rgb(255,0,0)']]
           ,opacity = 1
                      )
        ,text = data['summary']
        ,hovertemplate = '%{text} <extra></extra>'
        ))

# plot the number of ended cases.
fig.add_trace(go.Scattergeo(
         lat = data['Lat']
        ,lon = data['Long_']
        ,marker = dict(
            size = data['Ended-ref']
           ,color = data['Ended-ref'] 
           ,colorscale = 'Greens' 
           ,opacity = 1 
                      )
        ,text = data['summary']
        ,hovertemplate = '%{text} <extra></extra>'
        ))

# plot the number of recovered cases.
fig.add_trace(go.Scattergeo(
         lat = data['Lat']
        ,lon = data['Long_']
        ,marker = dict(
            size = data['Death-ref']
           ,color = 'black' 
           ,opacity = 1 
                      )
        ,text = data['summary']
        ,hovertemplate = '%{text} <extra></extra>'
        ))

# draw the border of each country.
fig.update_geos(
    showcountries=True
)

# show figure
fig.show()